In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

social = pd.read_csv("Aggregated_reddit_twitter.csv")
social = social.rename(columns={"date": "timestamp"})
social["timestamp"] = pd.to_datetime(social["timestamp"])

/tmp/ipykernel_176538/4215461173.py:5: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  social = pd.read_csv("Aggregated_reddit_twitter.csv")


In [3]:
social = social[social.content != "u/circular360"]
social = social[social.content != "There you go"]
social = social[
    social.content != "Why force the other player to stay around? Just give the winning player the option to continue the game with a perma passing AI controlling the surrendering player."]
# crypto = pd.read_csv("transformed_crypto.csv")
# crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)
crypto = pd.read_csv("crypto_differences.csv")
crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)

social = social.sort_values(by="timestamp")
# social.dtypes
sr = social.resample("T", on="timestamp").size()
freq = sr.reset_index()

freq = freq.rename(columns={0: "n_comments"})

In [4]:
freq["hour"] = freq["timestamp"].dt.hour
freq["day"] = freq["timestamp"].dt.dayofweek
freq["month"] = freq["timestamp"].dt.month
freq

,timestamp,n_comments,hour,day,month
0,2021-03-01 00:00:00,18,0,0,3
1,2021-03-01 00:01:00,8,0,0,3
2,2021-03-01 00:02:00,4,0,0,3
3,2021-03-01 00:03:00,11,0,0,3
4,2021-03-01 00:04:00,4,0,0,3
...,...,...,...,...,...
1132079,2023-04-26 03:59:00,0,3,2,4
1132080,2023-04-26 04:00:00,0,4,2,4
1132081,2023-04-26 04:01:00,0,4,2,4
1132082,2023-04-26 04:02:00,0,4,2,4


In [5]:
mg = pd.merge(crypto, freq, on="timestamp", how="inner")

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset


In [7]:
shortmg = mg.loc[0:5000]
shortmg.dropna()

,Unnamed: 0,timestamp,XRP-PERP,SOL-PERP,ETH-PERP,DOGE-PERP,BTC-PERP,APE-PERP,n_comments,hour,day,month


In [8]:
LAG = 30


class TimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)


class MyTimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data.loc[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)

# data = MyTimeSeriesDataset(X_data=shortmg[['n_comments', 'XRP-PERP', 'ETH-PERP']], y_data=shortmg['BTC-PERP'])

In [9]:
# Load your dataset into a pandas DataFrame called 'data'
from sklearn.preprocessing import MinMaxScaler

# Scale the features using MinMaxScaler
cs = ['XRP-PERP', 'ETH-PERP', 'BTC-PERP', 'SOL-PERP', 'DOGE-PERP']
scaled_data = shortmg[cs + ["hour", "day", "month", "n_comments"]]
# scaled_data = shortmg[['XRP-PERP', 'ETH-PERP', 'BTC-PERP', 'SOL-PERP', 'DOGE-PERP', 'n_comments']].to_numpy()


# to make classifier
# for x in cs:
#     scaled_data[x] = scaled_data[x].apply(lambda x: 1 if x > 0 else 0)

scaled_data = scaled_data.to_numpy()

scaler = MinMaxScaler()

scaled_data = scaler.fit_transform(scaled_data)
#
# Create sequences and targets for the RNN model
seq_length = 60
X_data = []
y_data = []

future = 0

for i in range(len(scaled_data) - seq_length - future):
    X_data.append(scaled_data[i:i + seq_length])
    y_data.append(scaled_data[i + seq_length + future])

y_data = np.delete(y_data, [-1, -2, -3, -4], axis=1)
# y_data = np.delete(y_data, [-1, -2, -3, -4, -5, -6, -7, -8], axis=1)
print(y_data[1])
X_data, y_data = np.array(X_data), np.array(y_data)

[0.47995546 0.6530303  0.52628435 0.55414013 0.28404537]


In [10]:
train_size = int(len(X_data) * 0.8)
X_train, X_test = X_data[:train_size], X_data[train_size:]
y_train, y_test = y_data[:train_size], y_data[train_size:]

In [100]:
from torch.utils.data import DataLoader

batch_size = 64
train_dataset = TimeSeriesDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
test_dataset = TimeSeriesDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [101]:
# classifier
# y_data = y_data.flatten()
# y_data.shape

In [102]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, output_size):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Linear(input_size, d_model)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers)
        self.fc = nn.Linear(d_model, output_size)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)
        x = self.fc(x[:, -1, :])
        return x



array([0.1486936])

In [103]:
from torch import optim
import torch.nn.functional as f


input_size = X_data.shape[2]
hidden_size = 256
num_layers = 8
output_size = y_data.shape[1]
# output_size = 1

model = RNNModel(input_size, hidden_size, num_layers, output_size)
# model = TransformerModel(input_size, d_model=32, nhead=2, num_layers=2, output_size=output_size)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [104]:
import platform

num_epochs = 30
log = False

# if "Linux" in platform.platform():
#     model = torch.compile(model)

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        y_pred = model(X_batch)

        # print(y_pred.flatten())
        # print(y_batch.shape.flatten())
        # loss = f.nll_loss(y_pred.flatten(), y_batch.flatten())
        loss = criterion(y_pred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 20 == 0 and log:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(X_batch), len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))

    model.eval()
    with torch.no_grad():
        test_loss = 0
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            test_loss += loss.item()

        test_loss = test_loss / len(test_loader)

        print(f"Epoch: {epoch + 1}, Train Loss: {loss.item():.4f}, Test Loss: {test_loss:.4f}")



Epoch: 1, Train Loss: 0.3052, Test Loss: 0.1561
Epoch: 2, Train Loss: 0.3081, Test Loss: 0.1560
Epoch: 3, Train Loss: 0.3075, Test Loss: 0.1559
Epoch: 4, Train Loss: 0.3067, Test Loss: 0.1565
Epoch: 5, Train Loss: 0.3058, Test Loss: 0.1564
Epoch: 6, Train Loss: 0.3125, Test Loss: 0.1602
Epoch: 7, Train Loss: 0.3109, Test Loss: 0.1582
Epoch: 8, Train Loss: 0.3070, Test Loss: 0.1559
Epoch: 9, Train Loss: 0.3074, Test Loss: 0.1569
Epoch: 10, Train Loss: 0.3058, Test Loss: 0.1557
Epoch: 11, Train Loss: 0.3033, Test Loss: 0.1557
Epoch: 12, Train Loss: 0.3085, Test Loss: 0.1569
Epoch: 13, Train Loss: 0.3135, Test Loss: 0.1581
Epoch: 14, Train Loss: 0.3061, Test Loss: 0.1553
Epoch: 15, Train Loss: 0.3112, Test Loss: 0.1578
